# Modelagem preditiva

A modelagem preditiva são modelos computacionais que usam a matemática para prever resultados futuros. Os algoritmos preditivos usam dados de treino para aprender sobre o comportamento das variáveis afim de buscar a aproximação da função alvo, a função alvo é a formulação matemática que explica a relação entre os dados de entrada e de saída.

A análise preditiva está sendo utilizada por empresas e indivíduos em todo o mundo para extrair valor de dados históricos, após ser desenvolvido e validado, o modelo preditivo consegue generalizar o conhecimento aprendido dos dados históricos para prever o futuro.

### Importando as bibliotecas necessárias

Para fazer nosso trabalho precisamos das funções e métodos por isso vamos importar as bibliotecas do python.

In [3]:
import pandas as pd
import numpy as np
import pickle

from tpot import TPOTRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score 

In [2]:
import warnings
warnings.filterwarnings('ignore')

### Carregando o conjunto de dados

In [66]:
# Carregando o conjunto de dados e salvando na variável 'imoveis'.
imoveis = pd.read_csv('../imoveis_clean_base.csv')

# Vamos tirar as colunas 'Unnamed: 0', 'Link', 'Estado', 'Cidade' e 'Descrição'
imoveis = imoveis.drop(['Unnamed: 0', 'Link', 'Estado', 'Descrição'], axis=1)

In [67]:
pd.set_option('display.max_columns', 100)

# Verificando o dataset
imoveis.head()

,Cidade,Valor_aluguel,Valor_condomínio,Valor_iptu,Área_total,Qt_quartos,Qt_vagas,Qt_banheiros,Academia,Acesso_para_deficientes,Ar_condicionado,Área_de_serviço,Armário_embutido,Armário_na_cozinha,Bicicletário,Churrasqueira,Circuito_de_segurança,Conexão_à_internet,Elevador,Espaço_gourmet,Garagem,Interfone,Lavanderia,Mobiliado,Piscina,Playground,Quadra_de_tênis,Quadra_poliesportiva,Salão_de_festas,Sauna,Segurança_24h,Sistema_de_alarme,Spa,Varanda
0,belo horizonte,3250.0,700.0,0.0,75.0,3.0,2.0,2.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,belo horizonte,3100.0,856.0,290.0,60.0,2.0,2.0,2.0,1,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,belo horizonte,1500.0,280.0,160.0,110.0,3.0,2.0,3.0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1
3,belo horizonte,1300.0,399.0,100.0,60.0,2.0,1.0,2.0,1,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,1,1,1,0,0,0,0
4,belo horizonte,1100.0,360.0,110.0,95.0,3.0,2.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0


In [68]:
# Verificando a dimensão do nosso dataset
imoveis.shape

(15958, 34)

In [69]:
df = imoveis

Nesse modelo vamos usar também a variável 'Cidade' porém como essa variável é do tipo texto temos que trata-lá para que o algoritmo possa receber ela da forma certa, em outras palavras como valor numérico. O pacote LabelEncoder transforma rótulos de texto em números.

In [70]:
cols = ["belo horizonte","porto-velho","manaus","rio-branco","campo-grande","macapa","brasilia","boa-vista",\
"cuiaba","palmas","sao-paulo","teresina","rio-de-janeiro","belem","goiania","salvador","florianopolis","sao-luis",\
"maceio","porto-alegre","curitiba","fortaleza","recife","joao-pessoa","aracaju","natal","vitoria"]

for i in cols:
    df[i] = 0

for i in cols:
    df[i] = np.where(df.Cidade == i, 1, 0)

In [71]:
df.shape

(15958, 61)

In [72]:
df.head()

,Cidade,Valor_aluguel,Valor_condomínio,Valor_iptu,Área_total,Qt_quartos,Qt_vagas,Qt_banheiros,Academia,Acesso_para_deficientes,Ar_condicionado,Área_de_serviço,Armário_embutido,Armário_na_cozinha,Bicicletário,Churrasqueira,Circuito_de_segurança,Conexão_à_internet,Elevador,Espaço_gourmet,Garagem,Interfone,Lavanderia,Mobiliado,Piscina,Playground,Quadra_de_tênis,Quadra_poliesportiva,Salão_de_festas,Sauna,Segurança_24h,Sistema_de_alarme,Spa,Varanda,belo horizonte,porto-velho,manaus,rio-branco,campo-grande,macapa,brasilia,boa-vista,cuiaba,palmas,sao-paulo,teresina,rio-de-janeiro,belem,goiania,salvador,florianopolis,sao-luis,maceio,porto-alegre,curitiba,fortaleza,recife,joao-pessoa,aracaju,natal,vitoria
0,belo horizonte,3250.0,700.0,0.0,75.0,3.0,2.0,2.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,belo horizonte,3100.0,856.0,290.0,60.0,2.0,2.0,2.0,1,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,belo horizonte,1500.0,280.0,160.0,110.0,3.0,2.0,3.0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,belo horizonte,1300.0,399.0,100.0,60.0,2.0,1.0,2.0,1,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,belo horizonte,1100.0,360.0,110.0,95.0,3.0,2.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [41]:
#labelencoder = LabelEncoder()
#df['Cidade'] = labelencoder.fit_transform(df['Cidade'])

In [73]:
# Separando a variável target
X = df.drop(['Valor_aluguel','Cidade'], axis=1)
y = df['Valor_aluguel'].values

In [74]:
X

,Valor_condomínio,Valor_iptu,Área_total,Qt_quartos,Qt_vagas,Qt_banheiros,Academia,Acesso_para_deficientes,Ar_condicionado,Área_de_serviço,Armário_embutido,Armário_na_cozinha,Bicicletário,Churrasqueira,Circuito_de_segurança,Conexão_à_internet,Elevador,Espaço_gourmet,Garagem,Interfone,Lavanderia,Mobiliado,Piscina,Playground,Quadra_de_tênis,Quadra_poliesportiva,Salão_de_festas,Sauna,Segurança_24h,Sistema_de_alarme,Spa,Varanda,belo horizonte,porto-velho,manaus,rio-branco,campo-grande,macapa,brasilia,boa-vista,cuiaba,palmas,sao-paulo,teresina,rio-de-janeiro,belem,goiania,salvador,florianopolis,sao-luis,maceio,porto-alegre,curitiba,fortaleza,recife,joao-pessoa,aracaju,natal,vitoria
0,700.0,0.0,75.0,3.0,2.0,2.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,856.0,290.0,60.0,2.0,2.0,2.0,1,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,280.0,160.0,110.0,3.0,2.0,3.0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,399.0,100.0,60.0,2.0,1.0,2.0,1,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,360.0,110.0,95.0,3.0,2.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15953,300.0,40.0,44.0,1.0,0.0,1.0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
15954,600.0,0.0,90.0,3.0,1.0,2.0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
15955,1200.0,0.0,52.0,2.0,1.0,2.0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
15956,350.0,0.0,35.0,1.0,1.0,1.0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [75]:
y

array([3250., 3100., 1500., ..., 1600., 1000., 3500.])

### Criando dados de treino e teste

In [76]:
# Separando dados de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

### Modelo com o Tpot AutoML

O objetivo do TPOT é automatizar a construção de pipelines de machine learning combinando uma representação de árvore de expressão flexível de pipelines com algoritmos de pesquisa. O TPOT usa a biblioteca scikit-learn baseada em Python.

In [81]:
model = TPOTRegressor(generations=2, 
                      population_size=50, 
                      scoring='r2', 
                      cv=3, verbosity=2, 
                      random_state=0)

In [82]:
%%time
model.fit(X_train, y_train)

[07:01:11] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:01:11] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:01:12] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:01:13] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:01:13] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:01:14] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:01:14] WARNI

[07:01:30] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:01:34] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:01:37] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:02:00] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:02:05] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:02:10] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:02:26] WARNI

[07:11:06] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:13:57] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:14:02] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:14:06] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Generation 1 - Current best internal CV score: 0.6765965954288045
[07:15:16] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:15:16] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linea

TPOTRegressor(config_dict=None, crossover_rate=0.1, cv=3,
              disable_update_check=False, early_stop=None, generations=2,
              max_eval_time_mins=5, max_time_mins=None, memory=None,
              mutation_rate=0.9, n_jobs=1, offspring_size=None,
              periodic_checkpoint_folder=None, population_size=50,
              random_state=0, scoring='r2', subsample=1.0,
              template='RandomTree', use_dask=False, verbosity=2,
              warm_start=False)

### Exportando a pipeline de execução

In [83]:
model.export('pipeline.py')
with open('pipeline.py') as pipe:
    print("".join(pipe.readlines()))

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from xgboost import XGBRegressor

# NOTE: Make sure that the class is labeled 'target' in the data file
tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
features = tpot_data.drop('target', axis=1).values
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['target'].values, random_state=0)

# Average CV score on the training set was:0.6773141099828616
exported_pipeline = make_pipeline(
    RobustScaler(),
    XGBRegressor(learning_rate=0.1, max_depth=6, min_child_weight=6, n_estimators=100, nthread=1, subsample=0.4)
)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)



### Executando a pipeline

In [88]:
model_tpot = GradientBoostingRegressor(alpha=0.85, learning_rate=0.1, loss="ls", max_depth=9,\
                                              max_features=1.0, min_samples_leaf=20, min_samples_split=18, \
                                              n_estimators=100, subsample=0.8500000000000001)

model_tpot.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.85, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=9,
                          max_features=1.0, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=20, min_samples_split=18,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=0.8500000000000001,
                          tol=0.0001, validation_fraction=0.1, verbose=0,
                          warm_start=False)

In [89]:
result = model_tpot.predict(X_test)
result

array([1762.30708135, 2173.8740601 , 1503.91028339, ...,  665.2887783 ,
       2920.75265142, 2942.25141692])

In [90]:
R2 = r2_score(y_test,result)
MAE = mean_absolute_error(y_test,result)
print(R2)
print(MAE)

0.6907816249732517
338.2112892480516


### Salvando o modelo

Para não precisar treinar o modelo em outros arquivos ou sistemas, podemos salvar nosso modelo com o pickle, isso faz com que possamos carregar nosso modelo em diferentes arquivos, sendo assim somente usar a função predict para fazer as previsões.

In [80]:
pickle.dump(model_tpot, open('model.sav', 'wb'))

------------------------------------

In [ ]:
# Cidades que não tem no dataset
#"rio-branco" "macapa" "boa-vista"

## Métricas de avaliação

### R-Quadrado (R²)

O R-Quadrado, ou Coeficiente de Determinação, é uma métrica que visa expressar a quantidade da variança dos dados que é explicada pelo modelo. Em outras palavras, essa medida calcula qual a porcentagem da variança que pôde ser prevista pelo modelo de regressão e, portanto, nos diz o quão “próximo” as medidas reais estão do nosso modelo.

O valor do seu R-Quadrado varia de 0 a 1 e geralmente é representado em porcentagem. Por exemplo, um R² = 75% nos diz que 75% da variância de nossos dados podem ser explicados pelo modelo construído, enquanto os outros 25%, teoricamente, se tratariam de uma variância residual.

### Erro Absoluto Médio (MAE)

O Erro Absoluto Médio consiste na média das distâncias entre valores preditos e reais. Em outras palavras, tira-se média da diferença absoluta dos valores preditos e real.

In [ ]:
# No exemplo abaixo vamos ver o MSE do algoritmo XGB, pois no loop for foi o último a ser executado, na matemática fica assim:
(abs(y_test - result).sum() / len(result))

### Erro Quadrático Médio (MSE)

O Erro Quadrático Médio consiste na média do erro das previsões ao quadrado. Em outras palavras, é a média da soma dos resíduos(diferença entra os valores predito e real) ao quadrado.

In [ ]:
# No exemplo abaixo vamos ver o MSE do algoritmo XGB, pois no loop for foi o último a ser executado, na matemática fica assim:
((y_test - result)**2).sum() / len(result)

---------------------------------------------------------------

### Melhor resultado do modelo:

In [56]:
R2 = r2_score(y_test,result)
MAE = mean_absolute_error(y_test,result)
print(R2)
print(MAE)

0.6858070355759733
341.10502070002883
